In [9]:
import pandas as pd
from pprint import pprint
from bsedata.bse import BSE
import os
from glob import glob
import sys
from datetime import datetime, timedelta, date
from path import Path
import time
#change
# from ..common.config import *
# from ..common.indicators import *
# from ..common.tradingview import *

In [10]:
print(f"Starting program: {sys.argv[0]} at {datetime.now().time()}")

Starting program: /Users/yash/Desktop/Trading/code/env/lib/python3.11/site-packages/ipykernel_launcher.py at 00:10:23.270477


In [11]:
end_date = datetime.today() - timedelta(days=1)
today_ = end_date.strftime("%Y/%m/%d")

#change
wl = Path("/Users/yash/Desktop/Trading/watchlists/")
path_ = wl / today_

if not os.path.isdir(path_):
    os.mkdir(path_)
os.chdir(path_)

In [12]:
final_outfile = datetime.today().strftime("%Y%m%d") + "_Q_IND_BSE.txt"

In [13]:
if final_outfile in glob("*IND*"):
    print("File already exists")
    exit()

In [14]:
#change
daily_ind = Path("/Users/yash/Desktop/Trading/historical_data/ind/daily/")

os.chdir(daily_ind)
files = glob("*.csv")

In [15]:
for f in files:
    if today_.replace("/", "") not in f:
        print("Deleting: ", f)
        os.remove(f)

In [16]:
b = BSE()
print(b)
b = BSE(update_codes = True)
scrips = b.getScripCodes()
print(f"Number of tickers in BSE: {len(scrips)}")

Driver Class for Bombay Stock Exchange (BSE)
Number of tickers in BSE: 4487


In [17]:
tickers = scrips.keys()
request_times = []
errors = 0
ticker_df = pd.DataFrame(columns=["Ticker", "1m", "3m", "6m", "ADV", "ADR%"])

In [18]:
def getHistoricalDataBSE(ticker, period = '6M'):
    his = b.getPeriodTrend(ticker,period)
    df = pd.DataFrame(his)
    # print(df.columns)
    df.rename(columns={"date": "Date", "value": "close", "vol": "volume"}, inplace=True)
    df['Date'] = df["Date"].apply(lambda x: datetime.strptime(x, "%a %b %d %Y %H:%M:%S").strftime("%d-%m-%Y"))
    return df

In [19]:
def n_month_gain(df, n):
    col_name = str(n) + "M_low"
    df[col_name] = df["close"].rolling(min(df.shape[0], n * 22)).min()
    # print(df.head())
    col_name2 = col_name + "_gain"
    df[col_name2] = (df["close"] / df[col_name] - 1) * 100
    df.drop(col_name, inplace=True, axis=1)
    return df

In [20]:
start_time = time.time()
for ticker in tickers:
    print(ticker)
    try:
        if not isinstance(ticker, str):
            print(f"Not a ticker {ticker}")
            continue

        outfile = ticker + "_" + today_.replace("/", "") + ".csv"
        if outfile in files:
            print("Ticker present: ", ticker)
            df = pd.read_csv(outfile)
        else:
            # # API LIMIT
            # while (
            #     len(request_times) >= 2000 and (time.time() - request_times[-2000]) < 3600
            # ):
            #     print("here")
            #     time.sleep(1)

            df = getHistoricalDataBSE(ticker)
            request_times.append(time.time())
            # ADD TAs
            adv_period = 20
            df["AvgVolume"] = df["volume"].rolling(adv_period).mean()
            df["AvgDollarVolume"] = df["close"] * df["AvgVolume"]
            # adr(df)
            for i in [1, 3, 6]:
                n_month_gain(df, i)

        last_row = df.iloc[-1]
        ticker_df.loc[len(ticker_df.index)] = [
            ticker,
            last_row["1M_low_gain"],
            last_row["3M_low_gain"],
            last_row["6M_low_gain"],
            last_row["AvgDollarVolume"],
            # last_row["ADR%"], #change
            0
        ]
        df.to_csv(outfile)
    except:
        print("Error for ticker: ", ticker)
        errors += 1
print("Time taken: {}".format(time.time() - start_time))

533022
532628
523395
590116
512161
538351
539300
530499
533292
531611
530027
531866
539096
538812
524412
524348
524208
514274
531731
519319
512038
523204
512165
500002
500488
505665
513119
520123
532682
532057
538952
532831
538935
511756
526955
531161
500410
532774
517494
532268
531897
526347
530513
530093
531525
536492
538570
517356
530901
530043
539391
533330
513149
524091
532762
511706
511359
512599
532921
533096
539254
538563
526711
523411
538734
507852
519183
539189
538365
532727
530117
531592
590088
532056
514113
500057
535755
532974
500303
522150
513513
521141
539056
530431
523120
517041
523031
521048
534612
506947
531047
517552
523269
532840
531429
531686
500003
530707
531921
500463
539042
516020
537292
537492
519281
500215
524448
532811
522273
532806
532683
506822
524288
532975
532331
519216
511692
530713
513349
526628
530621
538778
539017
532351
524598
500710
535916
539115
531517
531082
524075
531409
526707
532114
506235
533573
530973
517546
531156
505216
531147
531581
530889

In [24]:
pd.options.mode.chained_assignment = None  # default='warn'

In [45]:
# PARAMETERS
cr = 10**7
adr_filter = 3.5
dv_filter = 5 * cr
limit = 100

# %%
# FILTER
ticker_df_filt = ticker_df[
    # (ticker_df["ADR%"] >= adr_filter) & #change
    (ticker_df["ADV"] >= dv_filter)
]

# %%
# RANK
ticker_df_filt["1m_Rating"] = ticker_df_filt["1m"].rank(ascending=False)
ticker_df_filt["3m_Rating"] = ticker_df_filt["3m"].rank(ascending=False)
ticker_df_filt["6m_Rating"] = ticker_df_filt["6m"].rank(ascending=False)

ticker_df_1m = ticker_df_filt[ticker_df_filt["1m_Rating"] <= limit]
ticker_df_3m = ticker_df_filt[ticker_df_filt["3m_Rating"] <= limit]
ticker_df_6m = ticker_df_filt[ticker_df_filt["6m_Rating"] <= limit]

# %%
print(ticker_df_1m.shape[0])
print(ticker_df_3m.shape[0])
print(ticker_df_6m.shape[0])

# %%
ticker_df_final = pd.concat([ticker_df_1m, ticker_df_3m, ticker_df_6m], axis=0)
ticker_df_final = ticker_df_final.drop_duplicates(subset=["Ticker"], keep="first")

# %%
os.chdir(wl / today_)

# %%
s1 = set(ticker_df_1m["Ticker"])
set_to_tv(s1, datetime.today().strftime("%Y%m%d") + "_1_M_Q_IND_BSE.txt")
s3 = set(ticker_df_3m["Ticker"])
set_to_tv(s3, datetime.today().strftime("%Y%m%d") + "_3_M_Q_IND_BSE.txt")
s6 = set(ticker_df_6m["Ticker"])
set_to_tv(s6, datetime.today().strftime("%Y%m%d") + "_6_M_Q_IND_BSE.txt")

s = set(ticker_df_final["Ticker"])

100
100
100
20230613_1_M_Q_IND_BSE.txt
20230613_3_M_Q_IND_BSE.txt
20230613_6_M_Q_IND_BSE.txt


In [30]:
os.chdir("/Users/yash/Desktop/Trading/code/trading/q")

In [31]:
df_bse = pd.read_csv('BSE.csv')

In [33]:
df_bse['ticker_code'] = df_bse.index

In [42]:
df_bse['ticker_code'] = df_bse['ticker_code'].astype(str)

In [43]:
df_filt = df_bse[df_bse['ticker_code'].isin(s)]

In [46]:
s = set(df_filt["Issuer Name"])
print(len(s))
set_to_tv(s, final_outfile)

117
20230613_Q_IND_BSE.txt
